In [ ]:
# Lets try to run the ETL process locally, step by step

In [ ]:
# Import
import os
import sys
import configparser
import boto3
from datetime import datetime
import os
import pyspark.sql as Spark
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [ ]:
# Creates a spark session, or retrieve a matching one if it exists
spark = Spark.SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .getOrCreate()

In [ ]:
input_data = "./data/"
output_data = "./output/"

In [ ]:
# Execute unpack-local.sh 
# This unpacks the zip files in ./data/ and move to dirs matching S3 for the "real" etl.py

In [ ]:
# First up, songs data

In [ ]:
# read song data file
song_data = input_data + "song_data/*/*/*/*.json"
# Documentation: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader
# columnNameOfCorruptRecord: allows renaming the new field having malformed string created by PERMISSIVE mode.
# (DEFAULT) PERMISSIVE: To keep corrupt records, an user can set a string type field named columnNameOfCorruptRecord in an user-defined schema
df = spark.read.json(song_data, columnNameOfCorruptRecord='corrupt_record').drop_duplicates()

In [ ]:
# extract columns to create songs table
# Udacity instructions: 
# songs - songs in music database
# song_id, title, artist_id, year, duration
songs_table = df.select("song_id","title","artist_id","year","duration").drop_duplicates()

In [ ]:
# write songs table to parquet files partitioned by year and artist
songs_table.write.parquet(output_data + "songs/", 
                          mode="overwrite",
                          partitionBy=["year","artist_id"])

In [ ]:
# extract columns to create artists table
# Udacity instructions: 
# artists - artists in music database
# artist_id, name, location, lattitude, longitude
artists_table = df.select("artist_id","artist_name","artist_location","artist_latitude","artist_longitude").drop_duplicates()

In [ ]:
# write artists table to parquet files
artists_table.write.parquet(output_data + "artists/", 
                            mode="overwrite")

In [ ]:
# 